In [1]:
#Check Python Version
import shutil
import sys
sys.version
import os
cwd = os.getcwd()
print(cwd)
# import tfx_addons as tfxa

/usr/local/google/home/alexanderho/Repos/tfx-addons/examples/copy_tfrecords


In [2]:
#Upgrade pip
%pip install --upgrade pip
%pip install --upgrade pip
%pip install --upgrade "tfx[kfp]<2"
%pip uninstall tfx-addons

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))
# from custom_components.hello import component
# print(component)

2023-03-20 20:00:34.270706: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 20:00:34.447766: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-20 20:00:34.447826: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-20 20:00:35.753733: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

TensorFlow version: 2.11.0
TFX version: 1.12.0
KFP version: 1.8.19


In [4]:
GOOGLE_CLOUD_PROJECT = 'alex-tfx-dev'
GOOGLE_CLOUD_REGION = 'us-west1'
GCS_BUCKET_NAME = 'vertex-test-bucket-tfx'

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

In [5]:
PIPELINE_NAME = 'vertex-test'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))
print('DATA_ROOT: {}'.format(DATA_ROOT))

PIPELINE_ROOT: gs://vertex-test-bucket-tfx/pipeline_root/vertex-test
DATA_ROOT: gs://vertex-test-bucket-tfx/data/vertex-test


In [6]:
_trainer_module_file = 'penguin_trainer.py'

In [7]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils


from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _make_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Overwriting penguin_trainer.py


In [8]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

Copying file://penguin_trainer.py [Content-Type=text/x-python]...
/ [1 files][  3.8 KiB/  3.8 KiB]                                                
Operation completed over 1 objects/3.8 KiB.                                      


In [9]:
# Copyright 2019 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Example of a Hello World TFX custom component.

This custom component simply reads tf.Examples from input and passes through as
output.  This is meant to serve as a kind of starting point example for creating
custom components.

This component along with other custom component related code will only serve as
an example and will not be supported by TFX team.
"""
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
from typing import Dict, List
from tfx.v1.types.standard_artifacts import Examples
from tfx.dsl.component.experimental.decorators import component
from tfx.dsl.io import fileio

def _split_names_string_builder(split_names_list: List):
    str1 = "["
    urlist_len = len(split_names_list)-1
    index = 0

    for ele in split_names_list:
        if(index==urlist_len):
            str1 += "\""+ele+"\""+"]"
            break
        str1 += "\""+ele+"\""+","
        index+=1
    return str1

@component
def CopyExampleGen(
        input_dict: tfx.dsl.components.Parameter[Dict[str, str]],
        output_example: tfx.dsl.components.OutputArtifact[Examples]
      ) -> tfx.dsl.components.OutputDict():
    
        """Parse input_dict: creates a directory from the split-names and tfrecord uris provided"""
        split_names=[]
        for key in input_dict:
          split_names.append(key)
          
        split_names_string=_split_names_string_builder(split_names)
        output_example.split_names=str(split_names_string)
        
        """Make directories"""
        tfrecords_list=[]
        for key in input_dict:
          split_value="/Split-"+key+"/"
          fileio.mkdir(output_example.uri+split_value)
          tfrecords_list=fileio.glob(input_dict[key]+'*.gz')
          print("tfrecords_list: " + str(tfrecords_list))

          """"Copy files into directories"""
          for tfrecord in tfrecords_list:
                """TODO: Find a better way to extra file name"""
                file_name=os.path.basename(os.path.normpath(tfrecord))
                fileio.copy(tfrecord, output_example.uri+split_value+file_name, True)
                # print("file_name: "+file_name)

TensorFlow version: 2.11.0
TFX version: 1.12.0


In [10]:
# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple and
# slightly modified because we don't need `metadata_path` argument.
from tfx.utils import version_utils
from copy_example_gen import component

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     ) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  # example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  print('gcr.io/tfx-oss-public/tfx:{}'.format(version_utils.get_image_version()))

  # User input dictionary with split-names and their resepctive uri to tfrecords
  tfrecords_dict: Dict[str, str]={
    "train":'gs://vertex-test-bucket-tfx/examples/Split-train/',
    "eval":'gs://vertex-test-bucket-tfx/examples/Split-eval/',
  }

  copy_example = component.CopyExampleGen(
      input_dict=tfrecords_dict
  )
  # example_gen = tfx.components.CsvExampleGen(
  #   input_base=DATA_ROOT
  # )

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
    #   examples=copy_example.outputs['output_example'],
      examples=copy_example.outputs['output_example'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      copy_example,
      # examples_importer,
    #   copy_example,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

TensorFlow version: 2.11.0
TFX version: 1.12.0


In [11]:
# docs_infra: no_execute
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(default_image='gcr.io/tfx-oss-public/tfx:1.12.0'),
    output_filename=PIPELINE_DEFINITION_FILE)
# Following function will write the pipeline definition to PIPELINE_DEFINITION_FILE.
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        serving_model_dir=SERVING_MODEL_DIR))

gcr.io/tfx-oss-public/tfx:1.12.0
running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying penguin_trainer.py -> build/lib
installing to /tmp/tmpsshw4h5j
running install
running install_lib
copying build/lib/penguin_trainer.py -> /tmp/tmpsshw4h5j
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpsshw4h5j/tfx_user_code_Trainer-0.0+afa8d5dfaf2942b08385545b5e095850331200a5d7a669b1791b3e7efcc5fdd9-py3.8.egg-info
running install_scripts
creating /tmp/tmpsshw4h5j/tfx_user_code_Traine

In [12]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

Creating PipelineJob


INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


PipelineJob created. Resource name: projects/581894718279/locations/us-west1/pipelineJobs/vertex-test-20230320200049


INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/581894718279/locations/us-west1/pipelineJobs/vertex-test-20230320200049


To use this PipelineJob in another session:


INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:


pipeline_job = aiplatform.PipelineJob.get('projects/581894718279/locations/us-west1/pipelineJobs/vertex-test-20230320200049')


INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/581894718279/locations/us-west1/pipelineJobs/vertex-test-20230320200049')


View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-west1/pipelines/runs/vertex-test-20230320200049?project=581894718279


INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-west1/pipelines/runs/vertex-test-20230320200049?project=581894718279
